In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
df = pd.read_csv("C:/Users/julia/Downloads/teste_indicium_precificacao.csv")
print(df.head())
print(df.info())
print(df.describe())

In [ ]:
df.dropna(inplace=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['price'], bins=50, kde=True)
plt.title("Distribuição dos Preços")
plt.xlabel("Preço ($)")
plt.ylabel("Frequência")
plt.show()

In [ ]:
df_grouped = df.groupby("bairro")["price"].mean().sort_values().tail(30)
plt.figure(figsize=(16,8))
plt.xticks(rotation=45, ha="right") 
df_grouped.plot(kind='bar')
plt.title("Preço Médio por Bairro")
plt.xlabel("Bairro")
plt.ylabel("Preço Médio ($)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
respostas = """
2. 
a) Onde seria mais indicada a compra?
   Com base na análise exploratória, bairros com preços médios elevados e alta demanda são os mais indicados. Ademais, pode ser mais difícil manter taxas de ocupação altas se um bairro possuir muitos imóveis disponíveis e a atratividade também e diretamente afetada pela proximidade a pontos turísticos, transporte público e comércios. Manhattan apresenta os preços médios mais altos, indicando uma boa oportunidade para investidores que buscam alto retorno.

b) O número mínimo de noites e a disponibilidade ao longo do ano interferem no preço?
   Sim, propriedades com baixa exigência de noites mínimas e alta disponibilidade tendem a ter preços mais competitivos.

c) Existe algum padrão no texto do nome do local para lugares de mais alto valor?
   Nomes que contêm palavras como "luxo", "vista", "exclusivo" tendem a estar associados a preços mais altos. O comportamento também ocorre com palavras relacionadas à localizações estratégicas ou diferenciais do imóvel.

3. Explicação da previsão do preço:
   A previsão foi feita através da coleta e limpeza dos dados, incluindo tratamento de valores ausente e transformação de váriaveis; análise exploratória, visando entender a correlação entre variáveis e distribuição dos preços; seleção de features que impactam diretamente o preço; divisão dos dados entre treino e teste; treinamento do modelo e avaliação do modelo.
   Essa previsão foi feita utilizando machine learning, com foco em regressão, já que queremos prever um valor numérico contínuo.
   Foi utilizado um modelo de regressão (RandomForestRegressor) para prever o preço, o qual foi escolhido por ser um modelo robusto e capaz de capturar relações não lineares, além de ser resistente a overfitting e não exigir normalização dos dados, porém, ele também possui desvantagens, como maior lentidão para previsões em tempo real, já que é baseado em múltiplas árvores, e uma maior dificuldade para ser interpretado se comparado a modelos lineares.
   As variáveis latitude, longitude, mínimo de noites, número de reviews, reviews por mês, número de listagens por host e disponibilidade ao longo do ano foram selecionadas por sua influência no preço.
   A métrica escolhida para avaliar o modelo foi o R², que mede o quão bem o modelo explica a variação do preço, outras métricas usadas incluem, MAE, a média dos erros absolutos e MSE, a média dos erros ao quadrado. O R² foi escolhido pois permite avaliar o quanto o modelo consegue capturar das variações reais do preço.

4. Qual seria a sua sugestão de preço?
   Com base no modelo de regressão RandomForestRegressor e nos atributos do apartamento "Skylit Midtown Castle", a previsão de preço é de aproximadamente $225, o que está alinhado com a média do mercado para a região.


"""
print(respostas)

In [ ]:
features = ["latitude", "longitude", "minimo_noites", "numero_de_reviews", "reviews_por_mes", "calculado_host_listings_count", "disponibilidade_365"]
X = df[features]
y = df["price"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

In [ ]:
with open("modelo_precos.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
apartamento = np.array([[40.75362, -73.98377, 1, 45, 0.38, 2, 355]])
apartamento_scaled = scaler.transform(apartamento)
predicted_price = model.predict(apartamento_scaled)
print(f"Preço sugerido: ${predicted_price[0]:.2f}")